# 歐式買權評價 - 蒙地卡羅模擬 
   - 第一部份  透過蒙地卡羅模擬近似歐式買權價格
   - 第二部分  以Black - Scholes 模型驗證蒙地卡羅模擬的結果

# 第一部份  透過蒙地卡羅模擬近似歐式買權價格
- 根據Black-Scholes Model，假設股價的隨機過程為幾何布朗運動，則股價報酬率服從對數常態分配。

- 由均勻(0,1)分配中抽取n筆資料並標準化，根據中央極限定理(CLT)，標準化後會漸近標準常態分配，故可得到一個近似標準常態分配的亂數。
	再重複進行 I 次，便可得到 I 個猶如從標準常態分配抽出的樣本(亂數)

- 將服從標準常態分配的 I 筆樣本，代入Black-Scholes model 假設下的到期日股價公式，再減去履約價(K)，
	便可得到 I 個選擇權到期日收益(ST - K)

- 將 I 筆到期日收益加總並取平均，可計算出到期日選擇權平均收益( (ST - K) / I )，實際上可視為樣本平均。
	依據弱大數法則(WLLN)，當樣本數 I 趨近無限大時，樣本平均會收斂至母體平均，
	因此，當 I 夠大時，樣本平均「到期日選擇權平均收益( (ST - K) / I )」會近似母體平均「到期日選擇權期望收益」

- 將「到期日選擇權平均收益」折現後，就會得到以蒙地卡羅模擬近似出來的歐式買權價格。

## Step 1 導入所需套件

In [1]:
import math
import random
import numpy as np
from scipy.stats import norm  # 導入標準常態累積分配之套件

## Step 2  設定參數

In [2]:
S0 = 100      # 標的資產現在的價格
K = 105       # 履約價
T = 1         # 到期日(單位:年)
r = 0.05      # 無風險利率
sigma = 0.2   # 年化波動度

n = 100       # 樣本數 (在均勻分配之抽樣數)
I = 100000    # 模擬次數 (在標準常態分配之抽樣數)

## Step 3 定義函數

### (1) 標準常態分配

In [3]:
def normal_z(n) :   
    x_sum = 0 
    for i in range(n) :
        x_sum = x_sum + random.random()   # 從均勻分配中隨機抽取n個樣本，並計算樣本和
    x_bar = x_sum / n   # 樣本平均數
    var = 1/12   # 均勻分配母體變異數
    u = 0.5   # 均勻分配母體期望值
    z = (x_bar - u) / math.sqrt(var/n)   #標準化
    return z

### (2) 股票到期日價格

In [4]:
def price() :
    z = normal_z(n)
    ST = S0*math.exp((r-sigma**2/2)*T+sigma*math.sqrt(T)*z)   # 假設股票價格的隨機過程為幾何布朗運動可推得此結果
    return ST

### (3) 取最大值

In [5]:
def max_self(a,b) :
    if a >= b :
        return a
    else :
        return b

### (4) 歐式買權價格

In [6]:
def call(I) :   
    profit_list = []
    for i in range(I) :
        profit_temp = price()   # 用profit_temp暫存計算出的I個ST
        profit = max_self(profit_temp - K,0)   # 利用取最大值函數計算選擇權利潤 (利潤最小 = 0)
        profit_list.append(profit)   # 將所有計算出來的利潤新增到profit_list中
    C0 = (sum(profit_list) / I) * math.exp(-r*T)   # 計算利潤之平均數後再折現
    C0_2 = round(C0,2)   # 四捨五入至小數點第二位
    return C0_2

## Step 4 輸出結果

In [7]:
C0_M = call(I)  # 以蒙地卡羅計算之歐式買權價格

# 第二部份 以Black - Scholes 模型驗證蒙地卡羅模擬的結果

## Step 1 透過Black-Scholes Model計算歐式買權價格

In [8]:
#   設定參數   #

d1 = (np.log(S0/K) + (r+sigma**2/2)*T )/sigma*np.sqrt(T)
d2 = d1 - sigma*np.sqrt(T)      

N_d1 = norm.cdf(d1)  # 標準常態分配之累積機率
N_d2 = norm.cdf(d2)

#   Black-Scholes Model   #  

C0_B = S0*N_d1 - K*np.exp(-r*T)*N_d2  # 以BS Model 計算之歐式買權價格
C0_B2 = round(C0_B,2)   # 四捨五入至小數點第二位

## Step 2 計算兩種算法的差異

In [9]:
#   兩種計算方式之差異   #

diff = abs( round(C0_B2 - C0_M , 2) )

## Step 3 輸出最終結果

In [10]:
print("歐式買權價格(蒙地卡羅) : ", C0_M )
print("歐式買權價格(BS模型) : " , C0_B2 )
print("兩種計算方式之差異 :" , diff )

歐式買權價格(蒙地卡羅) :  8.08
歐式買權價格(BS模型) :  8.02
兩種計算方式之差異 : 0.06
